# Gym

##### 1.调用环境：

In [ ]:
import gym
env = gym.make('CartPole-v0')
env.reset()
for _ in range(100):
    env.render()
    env.step(env.action_space.sample()) # take a random action
env.close()

##### 2.自定义环境：

搭建一个5X5的网格环境gridworld.py，放到gym\envs\classic_control路径下
- termianl包括终点和陷阱，修改`self.terminal_rewards`即可修改地图
- 使用`set_state()`设置当前状态点

在gym\envs\classic_control下的初始化文件_init_.py中添加

In [ ]:
from gym.envs.classic_control.gridworld import GridWorldEnv


在gym\envs下的初始化文件_init_.py中添加

In [ ]:
register (
    id= 'GridWorld-v0',
    entry_point='gym.envs.classic_control:GridWorldEnv', 
    max_episode_steps=200, 
    reward_threshold=100.0,
)


调用GridWorld环境

In [ ]:
import gym
import random

env = gym.make('GridWorld-v0')
env.reset()
for _ in range(20):
    env.render()
    if _ == 0:
        print("init_state:{}".format(env.state))
    next_state, reward, is_terminal,info = env.step(env.actions[int(random.random()*len(env.actions))]) # take a random action
    print("next_state:{}, reward:{}, is_terminal:{}".format(next_state, reward, is_terminal))
env.close()